In [56]:
import pandas as pd
import numpy as np
import ast
import nltk

In [57]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

# Finding number of rows and columns in respective datasets
# print("Movies dataset - ",movies.shape)
# print("Movies dataset - ",credits.shape)

In [58]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [59]:
# Merge two datasets into one 
# Finding common Tags in both datasets ( movie_id , title)
movies = movies.merge(credits,on='title')
movies.shape

# After merge 
#print("Movies_(Final_Dataset) -",movies.shape)

(4809, 23)

In [60]:
# Tells about columns present in this dataset
#movies.info()

In [61]:
# Now we are sorting Tags that might be usefull while reccommending ''
# genres , id ( To fetch posters { When we represent this project}) ,keywords
# title (Not original title because 'title is always in en')
# Overview , cast , crew ( Only director )

# Rest of the data will not be used in our movies dataframe

In [62]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
#movies.head()

In [63]:
# Finding Missing data
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [64]:
# Drop the Row with missing data
movies.dropna(inplace=True)

In [65]:
# Check for duplicate rows
movies.duplicated().sum()

0

In [66]:
print(movies['genres'].unique())

['[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
 '[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]'
 '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]'
 ...
 '[{"id": 53, "name": "Thriller"}, {"id": 27, "name": "Horror"}, {"id": 35, "name": "Comedy"}]'
 '[{"id": 10769, "name": "Foreign"}, {"id": 53, "name": "Thriller"}]'
 '[{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}, {"id": 10770, "name": "TV Movie"}]']


In [67]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [68]:
# Converting the list from genres more elegent 

def convert(obj):# Obj is string that consists of list and dictonary type of python objects
    list_=[]; 
    for i in ast.literal_eval(obj): # Now Obj is converted to List from string (Automatically , because it was its python object equivalent.)
        list_.append(i['name']) # Also we only stored names from this object , this is what we want .
    return list_
    

In [69]:
def Find_Top_4(obj):# _Obj_ is string ( same as before )
    list_=[]; 
    iterator = 0
    for i in ast.literal_eval(obj): # Now _Obj_ is converted to List from string
        if iterator != 4: # Iterate only to find top 4
            list_.append(i['name']) # Only information about names is required
            iterator+=1
        else :
            break
    return list_
    

In [70]:
# output List of genres and keywords in list format
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [71]:
# We filter out top actors in movies by their real name not character name from cast 
# "character": "Jake Sully"
# "name": "Sam Worthington"
movies['cast'] = movies['cast'].apply(Find_Top_4)
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [72]:
# Now at last we Edit the crew { Director of movie }

In [73]:
def Fetch_Director(obj):
    list_ = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            list_.append(i['name'])
            break
    return list_

In [74]:
movies['crew'] = movies['crew'].apply(Fetch_Director)

In [75]:
# movies.head()
movies['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [76]:
#movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [77]:
def list_to_string(lst):
    if isinstance(lst, list):
        return ' '.join(lst)  # Join list items into a single string
    return str(lst)  # Convert non-list items to string

In [78]:
# Strip whitespace from all column names
movies.columns = movies.columns.str.strip()
movies['genres'] = movies['genres'].apply(list_to_string)
movies['keywords'] = movies['keywords'].apply(list_to_string)
movies['crew'] = movies['crew'].apply(list_to_string)
movies['cast'] = movies['cast'].apply(list_to_string)

In [79]:
movies['genres'] = movies['genres'].str.lower()
movies['keywords'] = movies['keywords'].str.lower()
movies['crew'] = movies['crew'].str.lower()
movies['cast'] = movies['cast'].str.lower()

In [80]:
movies['tags'] = ( movies['overview'] + ' ' +
                  movies['genres'] + ' ' +
                  movies['keywords'] + ' ' +
                  movies['crew'] + ' ' +
                  movies['cast']
                 )

In [81]:
# Now we create a new Dataframe which contains all the entries that we will require
Final_df = movies[['movie_id','title','tags']]
Final_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy science fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d james cameron sam worthington zoe saldana sigourney weaver stephen lang'

In [82]:
Final_df.loc[:, 'tags'] = Final_df['tags'].apply(lambda x: x.lower())

In [83]:
# https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Bag of words
cv = CountVectorizer(max_features=5000, stop_words='english')

In [84]:
vectors = cv.fit_transform(Final_df['tags']).toarray()

In [85]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [86]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [87]:
vectors.shape
# It is tuple

(4806, 5000)

In [88]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zooey', 'zucker', 'zwick'], dtype=object)

In [89]:
# Applying Stemming , finds out root word of common words { action , actions etc}
# Installing nltk
from nltk.stem.porter import PorterStemmer 
ps = PorterStemmer()

In [90]:
# Helper Function

def stemming(text_to_be_stemmed):
    list_ = []
    for i in text_to_be_stemmed.split():
        list_.append(ps.stem(i))
    return " ".join(list_)

In [91]:
Final_df.loc[:,'tags'] = Final_df['tags'].apply(stemming)

In [92]:
Final_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi scienc fiction cultur clash futur space war space coloni societi space travel futurist romanc space alien tribe alien planet cgi marin soldier battl love affair anti war power relat mind and soul 3d jame cameron sam worthington zoe saldana sigourney weaver stephen lang'

In [93]:
# We are done with vectorization 
# Next step is to calculate cosine distance { Not euclidian distance }
# using cosine_similarity() function of sklearn
#  ( More Distance ) =  ( More Dis_Similiarity )

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
similarity = cosine_similarity(vectors)

In [96]:
# Now the similarity matrix we have here will always have its diagonal = '0' , because every movie must be 100 % similiar to itself .
# Here '0' is index of movie
# sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]
# A list will be printed containing top 5 similiar movies to our selection

In [97]:
Final_df.loc[:, 'title'] = Final_df['title'].apply(lambda x: x.lower())

In [98]:
# enumerate() function adds a counter to an iterable, which is useful when you need the index of items as you iterate over them.

# If distances is a list like [0.5, 0.2, 0.7, 0.3], enumerate(distances) will return:
# [(0, 0.5), (1, 0.2), (2, 0.7), (3, 0.3)]

In [99]:
def recommend(movie):
    movie = movie.lower()
    if movie not in Final_df['title'].values:
        print(f"Movie '{movie}' not found in the database.")
        return
    # Here index[0] means when we search for title if there are multiple matches , it will pick up the first one .
    # movie_index = Final_df[Final_df['title'] == movie] make sure it finds title in dataframe .
    
    movie_index = Final_df[Final_df['title'] == movie].index[0]
    distances = similarity[movie_index]

    # print(f"Distances for {movie}: {distances}")  # Debugging line

    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    print("Recommended Movies: ")
    for i in movies_list:
        print(Final_df.iloc[i[0]].title)


In [100]:
# Example usage
recommend("Batman")

Recommended Movies: 
batman
batman & robin
batman returns
batman begins
the dark knight rises


In [101]:
import pickle

In [102]:
Final_df.to_dict()

{'movie_id': {0: 19995,
  1: 285,
  2: 206647,
  3: 49026,
  4: 49529,
  5: 559,
  6: 38757,
  7: 99861,
  8: 767,
  9: 209112,
  10: 1452,
  11: 10764,
  12: 58,
  13: 57201,
  14: 49521,
  15: 2454,
  16: 24428,
  17: 1865,
  18: 41154,
  19: 122917,
  20: 1930,
  21: 20662,
  22: 57158,
  23: 2268,
  24: 254,
  25: 597,
  26: 271110,
  27: 44833,
  28: 135397,
  29: 37724,
  30: 558,
  31: 68721,
  32: 12155,
  33: 36668,
  34: 62211,
  35: 8373,
  36: 91314,
  37: 68728,
  38: 102382,
  39: 20526,
  40: 49013,
  41: 44912,
  42: 10193,
  43: 534,
  44: 168259,
  45: 72190,
  46: 127585,
  47: 54138,
  48: 81005,
  49: 64682,
  50: 9543,
  51: 68726,
  52: 38356,
  53: 217,
  54: 105864,
  55: 62177,
  56: 188927,
  57: 10681,
  58: 5174,
  59: 14161,
  60: 17979,
  61: 76757,
  62: 258489,
  63: 411,
  64: 246655,
  65: 155,
  66: 14160,
  67: 15512,
  68: 1726,
  69: 44826,
  70: 8487,
  71: 1735,
  72: 297761,
  73: 2698,
  74: 137113,
  75: 9804,
  76: 14869,
  77: 150540,
  78:

In [103]:
pickle.dump(Final_df,open('movies_Dictionary.pkl','wb'))

In [104]:
Final_df['title']

0                                         avatar
1       pirates of the caribbean: at world's end
2                                        spectre
3                          the dark knight rises
4                                    john carter
                          ...                   
4804                                 el mariachi
4805                                   newlyweds
4806                   signed, sealed, delivered
4807                            shanghai calling
4808                           my date with drew
Name: title, Length: 4806, dtype: object

In [105]:
Final_df['title'].values

array(['avatar', "pirates of the caribbean: at world's end", 'spectre',
       ..., 'signed, sealed, delivered', 'shanghai calling',
       'my date with drew'], dtype=object)

In [106]:
pickle.dump(Final_df,open('similarity_data.pkl','wb'))